# 此程度对应了文章中的第三个实验，用于求解光滑区域上的Helmholtz方程

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import scipy.special as scp
from torch.autograd import Variable as v
import time

In [4]:
# 设置求解区域，样本点和求出精确解

# k          波数
# theta0     准确解平面波的方向
# sample_num 样本点数
# sample_x   边界样本点坐标
# sample_u_r 准确解实部
# sample_u_i 准确解虚部
# rprime     边界参数化方程在样本点处切向量
# int_n      样本点处的外法向量
k , theta0 = 4 , np.pi/7
k1,k2 = k*np.cos(theta0) , k*np.sin(theta0) 
sample_num = 800 

theta = torch.linspace(2*np.pi/sample_num,2*np.pi,sample_num).reshape(-1,1)
r = 9/20-1/9*torch.cos(5*theta)
x0,x1 = torch.cos(theta),torch.sin(theta)
sample_x = torch.cat((x0,x1),1)*r

sample_u_r = torch.cos(k1*sample_x[:,0]+k2*sample_x[:,1]); 
sample_u_i = torch.sin(k1*sample_x[:,0]+k2*sample_x[:,1]);

rprime = torch.cat((-x1*r+5/9*torch.sin(5*theta)*x0,x0*r+5/9*torch.sin(5*theta)*x1),1)
int_n = rprime@torch.Tensor([[0,-1],[1,0]])
int_n = int_n/((int_n*int_n).sum(axis=1).reshape(-1,1).sqrt())

In [6]:
# 设置积分矩阵

# G1_r    基本解的实部
# G1_i    基本解的虚部
# A1_r    积分矩阵的实部
# A1_i    积分矩阵的虚部
# KaparR2 Kapar奇异积分的权重
time11 = time.time()
G1_r = torch.zeros(sample_num,sample_num)
G1_i = torch.zeros(sample_num,sample_num)
for i in range(sample_num):
    for j in range(sample_num):
        d = sample_x[j,:]-sample_x[i,:]
        r0 = d.norm()
        G1_r[i,j] = -scp.hankel1(0,k*r0).imag/4
        G1_i[i,j] = scp.hankel1(0,k*r0).real/4
KaparR2 = torch.zeros(sample_num,1)
KaparR2[0],KaparR2[1] = 1.825748064736159,-1.325748064736159
A1_r = torch.zeros(sample_num,sample_num)
A1_i = torch.zeros(sample_num,sample_num)
for i in range(sample_num):
    for j in range(sample_num):
        if i==j:
            A1_r[i,j],A1_i[i,j] = 0,0
        else:
            c = (1+KaparR2[abs(((i-j)+round(sample_num/2))%sample_num-round(sample_num/2)-1)])*((rprime[j,:]).norm())/sample_num*2*np.pi
            A1_r[i,j],A1_i[i,j] = -c*G1_r[i,j],-c*G1_i[i,j]
time12 = time.time()

In [11]:
# 构造网络和损失函数

# 全连接网络
# m 神经元个数
# Helmholtz 方程中输出是2维，分别代表实部和虚部
class Net(nn.Module):
  def __init__(self,m):
    super(Net, self).__init__()
    self.net=nn.Sequential(
      nn.Linear(in_features=2,out_features=m),nn.Tanh(),
      nn.Linear(m,m),nn.Tanh(),
      nn.Linear(m,m),nn.Tanh(),
      nn.Linear(m,m),nn.Tanh(),
      nn.Linear(m,m),nn.Tanh(),
      nn.Linear(m,2)
    )
  def forward(self, input:torch.FloatTensor):
    return self.net(input)

# MSE loss    
class Green_loss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,u_exact_r,u_exact_i,u_Green_r,u_Green_i):
        return torch.mean(torch.pow((u_exact_r-u_Green_r),2)) +torch.mean(torch.pow((u_exact_i-u_Green_i),2)) 

net1 = Net(40) #用格林函数形式作为损失函数
Green_loss_func = Green_loss()

In [15]:
#训练过程

# optimizer   优化器
# Epoch       总训练次数
# sample_h_r  样本点处密度函数值h(x)的实部
# sample_h_i  样本点处密度函数值h(x)的虚部
# u0_r        样本点处数值解实部
# u0_i        样本点处数值解虚部
optimizer = torch.optim.Adam(net1.parameters(net1),lr=0.001)
Epoch = 1000
time0 = time.time()
for epoch in range(Epoch+1):
    sample_h_r,sample_h_i = net1(sample_x)[:,0],net1(sample_x)[:,1]#这个方法甚至不需要内部点
    
    u0_r = (A1_r@sample_h_r-A1_i@sample_h_i)
    u0_i = (A1_r@sample_h_i+A1_i@sample_h_r)

    loss = Green_loss_func(sample_u_r,sample_u_i,u0_r,u0_i)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch%1000==0:
        print('loss, epoch, computation time:','%.4f'%loss.detach().numpy(),epoch,'%.4f'%(time.time()-time0))
        time0 = time.time()


loss, epoch, computation time: 0.0162 0 0.0399
loss, epoch, computation time: 0.0015 1000 21.8296


In [16]:
# 在区域内部测试精度

# sample 内部采样点数
# x_in   内部采样点坐标
# u_in_r 内部点准确解实部 
# u_in_i 内部点准确解虚部 
# G_in_r 内部点对应积分矩阵实部
# G_in_i 内部点对应积分矩阵虚部
sample = 1000
theta_in = torch.rand(sample,1)*2*np.pi
r_in = torch.rand(sample,1)*(9/20-1/9*torch.cos(5*theta_in))*0.99
x0_in = torch.cos(theta_in)
x1_in = torch.sin(theta_in)
x_in = torch.cat((x0_in,x1_in),1)*r_in

u_in_r = torch.cos(k1*x_in[:,0]+k2*x_in[:,1]); 
u_in_i = torch.sin(k1*x_in[:,0]+k2*x_in[:,1]);

G1_in_r = torch.zeros(sample,sample_num)
G1_in_i = torch.zeros(sample,sample_num)
for i in range(sample):
    for j in range(sample_num):
        d = sample_x[j,:]-x_in[i,:]
        r0 = d.norm()
        c_in = (rprime[j,:].norm())/sample_num*2*np.pi
        G1_in_r[i,j] = -scp.hankel1(0,k*r0).imag/4*c_in
        G1_in_i[i,j] = scp.hankel1(0,k*r0).real/4*c_in

In [17]:
# BINet求得的数值解实部和虚部
u_green_r = -(G1_in_r@sample_h_r-G1_in_i@sample_h_i)
u_green_i = -(G1_in_r@sample_h_i+G1_in_i@sample_h_r)

# 打印相对L2误差
print('Relaive L2 error:','%.4f'%(((((u_green_r-u_in_r)**2).sum()+((u_green_r-u_in_r)**2).sum())/((u_in_r**2).sum()+(u_in_r**2).sum())).sqrt()))

Relaive L2 error: 0.0110
